In [1]:
import requests

# 目标网址
url = "https://www.icourse163.org/course/NHDX-1463126169"

# 设置请求头，模拟浏览器访问
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# 发起请求
response = requests.get(url, headers=headers)

# 将响应的内容保存为HTML文件
file_path = 'icourse163_page.html'
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(response.text)

file_path


'icourse163_page.html'

In [2]:
!pip install selenium

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
from selenium import webdriver
from time import sleep

# 使用 Selenium Manager 自动管理 ChromeDriver
driver = webdriver.Chrome()
# driver = webdriver.Firefox()

# 打开目标网页
driver.get("https://www.icourse163.org/course/NHDX-1463126169")

# 等待页面加载
sleep(5)

# 保存网页源代码
with open("icourse163_selenium_saved.html", "w", encoding="utf-8") as file:
    file.write(driver.page_source)
    

driver.quit()


TypeError: write() argument must be str, not None

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 初始化 WebDriver，启动 Chrome 浏览器
driver = webdriver.Chrome()

# 打开目标网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 等待“课程评价”按钮加载，并点击
try:
    # 等待按钮可点击
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击“课程评价”按钮
    print("点击了'课程评价'按钮")
except Exception as e:
    print(f"未能找到或点击按钮，错误: {e}")

# 保存当前页面的 HTML 源代码
time.sleep(5)  # 等待页面完全加载
file_name = "course_reviews.html"
with open(file_name, "w", encoding="utf-8") as file:
    file.write(driver.page_source)  # 将页面源码写入文件
print(f"页面已保存为 {file_name}")

# 关闭浏览器
driver.quit()
print("浏览器已关闭")


点击了'课程评价'按钮
页面已保存为 course_reviews.html
浏览器已关闭


In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 创建一个 Chrome 浏览器实例
driver = webdriver.Chrome()


# 用于存储所有评论数据的列表
all_reviews = []

# 打开指定的网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 点击“课程评价”按钮，等待按钮可点击
try:
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击按钮
    time.sleep(3)  # 等待页面加载
except Exception as e:
    print(f"无法点击课程评价按钮: {e}")

# 定义一个函数用于提取页面上的评论内容
def extract_reviews():
    # 查找页面上的所有评论元素
    review_elements = driver.find_elements(By.XPATH, '//div[@class="ux-mooc-comment-course-comment_comment-list_item"]')
    
    # 遍历每个评论元素
    for review_element in review_elements:
        review_data = {}
        try:
            # 提取用户名称
            review_data['user'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_user-info_name'
            ).text

            # 提取评论时间
            review_data['time'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time'
            ).text

            # 提取评分（星星数）
            stars = review_element.find_elements(
                By.CLASS_NAME, 'ux-icon-custom-rating-favorite'
            )
            review_data['rating'] = len(stars)  # 星星数表示评分

            # 提取评论内容
            review_data['content'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_content'
            ).text
        except Exception as e:
            print(f"解析评论时出错: {e}")

        # 将每个评论数据添加到总列表中
        all_reviews.append(review_data)

# 定义一个函数用于点击下一页按钮
def next_page():
    try:
        # 查找并点击下一页按钮
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'))
        )
        next_button.click()  # 点击下一页按钮
        time.sleep(3)  # 等待页面加载
        return True  # 成功点击下一页按钮
    except Exception as e:
        print(f"无法点击下一页: {e}")
        return False  # 无法点击下一页按钮

# 提取指定页数的评论
num_pages = 10  # 设置要提取的页数
for page in range(1, num_pages + 1):
    print(f"正在提取第 {page} 页的评论...")
    extract_reviews()  # 提取当前页面的评论

    # 如果不是最后一页，尝试翻页
    if not next_page():
        print("已到达最后一页，停止翻页。")
        break

# 将所有提取的评论数据保存为 CSV 文件
df = pd.DataFrame(all_reviews)
output_file = "course_reviews1.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"数据已保存到 {output_file}")

# 关闭浏览器
driver.quit()
print("所有页面处理完毕，浏览器已关闭")


正在提取第 1 页的评论...
正在提取第 2 页的评论...
正在提取第 3 页的评论...
正在提取第 4 页的评论...
正在提取第 5 页的评论...
正在提取第 6 页的评论...
正在提取第 7 页的评论...
正在提取第 8 页的评论...
正在提取第 9 页的评论...
正在提取第 10 页的评论...
数据已保存到 course_reviews1.csv
所有页面处理完毕，浏览器已关闭


In [7]:
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.options import Options

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class MOOCReviewScraper:
    """中国大学MOOC课程评论爬取器"""
    
    def __init__(self, headless=False):
        """
        初始化爬虫
        :param headless: 是否使用无头模式
        """
        self.driver = None
        self.wait = None
        self.headless = headless
        
    def setup_driver(self):
        """设置Chrome驱动"""
        options = Options()
        if self.headless:
            options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        
        # 设置窗口大小
        options.add_argument('--window-size=1920,1080')
        
        try:
            self.driver = webdriver.Chrome(options=options)
            self.wait = WebDriverWait(self.driver, 10)
            logger.info("Chrome驱动初始化成功")
        except Exception as e:
            logger.error(f"Chrome驱动初始化失败: {e}")
            raise
    
    def navigate_to_reviews(self, course_url):
        """
        导航到课程评论页面
        :param course_url: 课程URL
        """
        try:
            logger.info(f"正在访问课程页面: {course_url}")
            self.driver.get(course_url)
            
            # 等待页面加载完成
            time.sleep(3)
            
            # 点击评论标签
            review_button = self.wait.until(
                EC.element_to_be_clickable((By.ID, "review-tag-num"))
            )
            review_button.click()
            logger.info("成功进入评论页面")
            
        except TimeoutException:
            logger.error("页面加载超时")
            raise
        except Exception as e:
            logger.error(f"导航到评论页面失败: {e}")
            raise
    
    def scrape_current_page_reviews(self):
        """抓取当前页面的所有评论"""
        reviews = []
        try:
            # 等待评论加载
            time.sleep(2)
            
            # 查找所有评论元素
            comment_elements = self.driver.find_elements(
                By.CSS_SELECTOR, 
                ".ux-mooc-comment-course-comment_comment-list_item"
            )
            
            for comment in comment_elements:
                try:
                    # 提取评论内容
                    content = comment.find_element(
                        By.CSS_SELECTOR, 
                        ".ux-mooc-comment-course-comment_comment-list_item_body"
                    ).text
                    
                    # 提取评论者信息（如果有）
                    try:
                        reviewer = comment.find_element(
                            By.CSS_SELECTOR,
                            ".ux-mooc-comment-course-comment_comment-list_item_author"
                        ).text
                    except:
                        reviewer = "匿名用户"
                    
                    # 提取评论时间（如果有）
                    try:
                        review_time = comment.find_element(
                            By.CSS_SELECTOR,
                            ".ux-mooc-comment-course-comment_comment-list_item_time"
                        ).text
                    except:
                        review_time = "未知时间"
                    
                    reviews.append({
                        'reviewer': reviewer,
                        'content': content,
                        'time': review_time
                    })
                    
                except Exception as e:
                    logger.warning(f"提取单条评论失败: {e}")
                    continue
            
            logger.info(f"当前页面抓取到 {len(reviews)} 条评论")
            return reviews
            
        except Exception as e:
            logger.error(f"抓取评论失败: {e}")
            return reviews
    
    def click_next_page(self):
        """点击下一页"""
        try:
            # 滚动到页面底部
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            
            # 查找并点击下一页按钮
            next_button = self.wait.until(
                EC.element_to_be_clickable((By.LINK_TEXT, "下一页"))
            )
            next_button.click()
            logger.info("成功点击下一页")
            return True
            
        except TimeoutException:
            logger.info("已到达最后一页")
            return False
        except Exception as e:
            logger.error(f"点击下一页失败: {e}")
            return False
    
    def scrape_all_reviews(self, course_url, max_pages=None):
        """
        抓取所有评论
        :param course_url: 课程URL
        :param max_pages: 最大抓取页数，None表示抓取所有页
        :return: 所有评论列表
        """
        all_reviews = []
        page_count = 0
        
        try:
            # 设置驱动
            self.setup_driver()
            
            # 导航到评论页面
            self.navigate_to_reviews(course_url)
            
            while True:
                page_count += 1
                logger.info(f"正在抓取第 {page_count} 页")
                
                # 抓取当前页评论
                page_reviews = self.scrape_current_page_reviews()
                all_reviews.extend(page_reviews)
                
                # 检查是否达到最大页数
                if max_pages and page_count >= max_pages:
                    logger.info(f"已达到最大抓取页数: {max_pages}")
                    break
                
                # 尝试点击下一页
                if not self.click_next_page():
                    break
                
                # 等待页面加载
                time.sleep(2)
            
            logger.info(f"抓取完成，共获取 {len(all_reviews)} 条评论")
            return all_reviews
            
        except Exception as e:
            logger.error(f"抓取过程出错: {e}")
            return all_reviews
        finally:
            self.cleanup()
    
    def save_reviews_to_file(self, reviews, filename="reviews.txt"):
        """
        保存评论到文件
        :param reviews: 评论列表
        :param filename: 文件名
        """
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                for i, review in enumerate(reviews, 1):
                    f.write(f"===== 评论 {i} =====\n")
                    f.write(f"评论者: {review['reviewer']}\n")
                    f.write(f"时间: {review['time']}\n")
                    f.write(f"内容: {review['content']}\n")
                    f.write("\n")
            logger.info(f"评论已保存到 {filename}")
        except Exception as e:
            logger.error(f"保存评论失败: {e}")
    
    def cleanup(self):
        """清理资源"""
        if self.driver:
            self.driver.quit()
            logger.info("浏览器已关闭")


def main():
    """主函数"""
    # 课程URL
    course_url = "https://www.icourse163.org/course/NHDX-1463126169"
    
    # 创建爬虫实例
    scraper = MOOCReviewScraper(headless=False)  # 设置为True可以使用无头模式
    
    try:
        # 抓取所有评论（可以设置max_pages限制页数）
        reviews = scraper.scrape_all_reviews(course_url, max_pages=5)  # 只抓取前5页
        
        # 保存评论到文件
        if reviews:
            scraper.save_reviews_to_file(reviews, "mooc_reviews.txt")
            
            # 打印部分评论
            print(f"\n共抓取到 {len(reviews)} 条评论")
            print("\n前5条评论预览:")
            for i, review in enumerate(reviews[:5], 1):
                print(f"\n--- 评论 {i} ---")
                print(f"评论者: {review['reviewer']}")
                print(f"时间: {review['time']}")
                print(f"内容: {review['content'][:100]}...")  # 只显示前100个字符
        else:
            print("未抓取到任何评论")
            
    except Exception as e:
        logger.error(f"程序执行失败: {e}")


if __name__ == "__main__":
    main()

2025-07-03 10:22:40,090 - INFO - Chrome驱动初始化成功
2025-07-03 10:22:40,091 - INFO - 正在访问课程页面: https://www.icourse163.org/course/NHDX-1463126169
2025-07-03 10:22:43,545 - INFO - 成功进入评论页面
2025-07-03 10:22:43,545 - INFO - 正在抓取第 1 页
2025-07-03 10:22:45,811 - INFO - 当前页面抓取到 20 条评论
2025-07-03 10:22:46,879 - INFO - 成功点击下一页
2025-07-03 10:22:48,883 - INFO - 正在抓取第 2 页
2025-07-03 10:22:51,139 - INFO - 当前页面抓取到 20 条评论
2025-07-03 10:22:52,194 - INFO - 成功点击下一页
2025-07-03 10:22:54,197 - INFO - 正在抓取第 3 页
2025-07-03 10:22:56,444 - INFO - 当前页面抓取到 20 条评论
2025-07-03 10:22:57,491 - INFO - 成功点击下一页
2025-07-03 10:22:59,497 - INFO - 正在抓取第 4 页
2025-07-03 10:23:01,741 - INFO - 当前页面抓取到 20 条评论
2025-07-03 10:23:02,801 - INFO - 成功点击下一页
2025-07-03 10:23:04,805 - INFO - 正在抓取第 5 页
2025-07-03 10:23:07,047 - INFO - 当前页面抓取到 20 条评论
2025-07-03 10:23:07,048 - INFO - 已达到最大抓取页数: 5
2025-07-03 10:23:07,048 - INFO - 抓取完成，共获取 100 条评论
2025-07-03 10:23:07,132 - INFO - 浏览器已关闭
2025-07-03 10:23:07,133 - INFO - 评论已保存到 mooc_reviews.txt



共抓取到 100 条评论

前5条评论预览:

--- 评论 1 ---
评论者: 匿名用户
时间: 未知时间
内容: jjxl2018
我是一名医学基础课程老师，负责病理生理学的教学，为了适应新时代的需求，我们学校开设了一个新兴医学专业“医 X”。在新医科以人工智能、大数据、云计算为代表的科技革命新背景下，本专业在病...

--- 评论 2 ---
评论者: 匿名用户
时间: 未知时间
内容: 君辞2560
非常好课程，使我的大脑旋转
发表于 2023-12-18
第6次开课
11...

--- 评论 3 ---
评论者: 匿名用户
时间: 未知时间
内容: mooc56490986133536953
老师讲的太好了吧，人美又温柔，耐心解决问题，我学会了，yyds
发表于 2021-11-02
第2次开课
7...

--- 评论 4 ---
评论者: 匿名用户
时间: 未知时间
内容: mooc110061795291999110
老师讲课生动有趣，带我们走进爬虫，让我又对计算机提起了兴趣(/≧▽≦)/~┴┴ (开心起来掀个桌没毛病吧～)
发表于 2022-03-24
第3次开课
6...

--- 评论 5 ---
评论者: 匿名用户
时间: 未知时间
内容: 乳酸钠林格
非常不错的课程，虽说是线上课程，但是学习效果丝毫不比线下课得效果差，老师线上回复十分及时，快速上手Python简直不要太爽，可视化的学习也是对Python语言理解的加深，更深一步理解了市面...


如果你浏览器有问题 可以用其他浏览器代替
self.driver = webdriver.Firefox()
将 Chrome WebDriver 换成 Firefox WebDriver。
。

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 创建一个 Firefox 浏览器实例
driver = webdriver.Firefox()  # 使用Firefox代替Chrome

# 用于存储所有评论数据的列表
all_reviews = []

# 打开指定的网页
url = "https://www.icourse163.org/course/NHDX-1463126169"
driver.get(url)

# 设置浏览器窗口大小
driver.set_window_size(1082, 672)

# 点击“课程评价”按钮，等待按钮可点击
try:
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "review-tag-button"))
    )
    review_button.click()  # 点击按钮
    time.sleep(3)  # 等待页面加载
except Exception as e:
    print(f"无法点击课程评价按钮: {e}")

# 定义一个函数用于提取页面上的评论内容
def extract_reviews():
    # 查找页面上的所有评论元素
    review_elements = driver.find_elements(By.XPATH, '//div[@class="ux-mooc-comment-course-comment_comment-list_item"]')
    
    # 遍历每个评论元素
    for review_element in review_elements:
        review_data = {}
        try:
            # 提取用户名称
            review_data['user'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_user-info_name'
            ).text

            # 提取评论时间
            review_data['time'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time'
            ).text

            # 提取评分（星星数）
            stars = review_element.find_elements(
                By.CLASS_NAME, 'ux-icon-custom-rating-favorite'
            )
            review_data['rating'] = len(stars)  # 星星数表示评分

            # 提取评论内容
            review_data['content'] = review_element.find_element(
                By.CLASS_NAME, 'ux-mooc-comment-course-comment_comment-list_item_body_content'
            ).text
        except Exception as e:
            print(f"解析评论时出错: {e}")

        # 将每个评论数据添加到总列表中
        all_reviews.append(review_data)

# 定义一个函数用于点击下一页按钮
def next_page():
    try:
        # 查找并点击下一页按钮
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'))
        )
        next_button.click()  # 点击下一页按钮
        time.sleep(3)  # 等待页面加载
        return True  # 成功点击下一页按钮
    except Exception as e:
        print(f"无法点击下一页: {e}")
        return False  # 无法点击下一页按钮

# 提取指定页数的评论
num_pages = 5  # 设置要提取的页数
for page in range(1, num_pages + 1):
    print(f"正在提取第 {page} 页的评论...")
    extract_reviews()  # 提取当前页面的评论

    # 如果不是最后一页，尝试翻页
    if not next_page():
        print("已到达最后一页，停止翻页。")
        break

# 将所有提取的评论数据保存为 CSV 文件
df = pd.DataFrame(all_reviews)
output_file = "course_reviewsfox.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"数据已保存到 {output_file}")

# 关闭浏览器
driver.quit()
print("所有页面处理完毕，浏览器已关闭")


正在提取第 1 页的评论...
正在提取第 2 页的评论...
正在提取第 3 页的评论...
正在提取第 4 页的评论...
正在提取第 5 页的评论...
数据已保存到 course_reviewsfox.csv
所有页面处理完毕，浏览器已关闭


In [6]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()

    # 创建一个列表来存储所有评论
    all_comments = []

    # 定义一个函数来收集当前页面的评论
    def collect_comments_on_current_page():
        # 等待评论加载
        time.sleep(2)

        # 获取当前页面上的所有评论
        comments = driver.find_elements(By.CSS_SELECTOR, ".m-reviewitm")

        # 提取每条评论的内容
        for comment in comments:
            try:
                # 评论者名称
                username = comment.find_element(By.CSS_SELECTOR, ".f-thide").text

                # 评论内容
                content = comment.find_element(By.CSS_SELECTOR, ".cnt").text

                # 评论时间
                time_element = comment.find_element(By.CSS_SELECTOR, ".time")
                comment_time = time_element.text

                # 创建评论字典
                comment_dict = {
                    "username": username,
                    "content": content,
                    "time": comment_time
                }

                # 添加到评论列表
                all_comments.append(comment_dict)
                print(f"已收集评论: {username} - {content}")

            except Exception as e:
                print(f"收集评论时出错: {e}")

    # 浏览多个页面并收集评论
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 收集第一页的评论
    collect_comments_on_current_page()

    # 导航到其他页面并收集评论
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 收集当前页面的评论
            collect_comments_on_current_page()

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    # 将评论保存到JSON文件
    with open("icourse_comments.json", "w", encoding="utf-8") as f:
        json.dump(all_comments, f, ensure_ascii=False, indent=4)

    print(f"成功收集了 {len(all_comments)} 条评论并保存到 icourse_comments.json")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

成功收集了 0 条评论并保存到 icourse_comments.json


In [5]:
pip install webdriver_manager

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 创建保存页面内容的目录
    save_dir = "icourse_pages"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()
    time.sleep(2)  # 等待评论区完全加载

    # 保存当前页面(第1页)评论区内容
    comment_area = driver.find_element(By.CSS_SELECTOR, ".m-review")
    comment_html = comment_area.get_attribute('outerHTML')

    # 保存第1页HTML内容
    with open(os.path.join(save_dir, "page_1.html"), "w", encoding="utf-8") as f:
        f.write(comment_html)
    print(f"已保存第1页评论区内容")

    # 页面索引计数器
    page_index = 2

    # 浏览多个页面并保存内容
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 导航到其他页面并保存评论区内容
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 保存当前页面评论区内容
            comment_area = driver.find_element(By.CSS_SELECTOR, ".m-review")
            comment_html = comment_area.get_attribute('outerHTML')

            # 保存当前页HTML内容
            with open(os.path.join(save_dir, f"page_{page_index}.html"), "w", encoding="utf-8") as f:
                f.write(comment_html)
            print(f"已保存第{page_index}页评论区内容")

            # 递增页面索引
            page_index += 1

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    print(f"成功保存了 {page_index-1} 个页面的评论区内容到 {save_dir} 目录")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

执行过程中出错: Message: no such element: Unable to locate element: {"method":"css selector","selector":".m-review"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104beaa54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000104be2cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x000000010472e864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000104775410 cxxbridge1$string$len + 382736
4   chromedriver                        0x00000001047b6480 cxxbridge1$string$len + 649088
5   chromedriver                        0x00000001047697ec cxxbridge1$string$len + 334572
6   chromedriver                        0x0000000104bafccc cxxbridge1$str$ptr + 2562928
7   chromedriver                        0x0000000104bb2f98 cxxbridge1

In [9]:
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    # 创建保存页面内容的目录
    save_dir = "icourse_pages"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 访问网页
    driver.get("https://www.icourse163.org/course/NHDX-1463126169")
    driver.set_window_size(1728, 905)

    # 等待评论区加载完成
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "review-tag-num"))
    )

    # 点击评论区
    driver.find_element(By.ID, "review-tag-num").click()
    time.sleep(5)  # 增加等待时间，确保评论区完全加载

    # 等待评论区出现
    try:
        comment_area = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".m-cmt"))
        )
    except:
        # 如果找不到.m-cmt，尝试其他可能的选择器
        possible_selectors = [".m-review", ".m-cmt", ".review-content", "#review-content", ".comment-list"]
        comment_area = None

        for selector in possible_selectors:
            try:
                comment_area = driver.find_element(By.CSS_SELECTOR, selector)
                print(f"找到评论区，使用选择器: {selector}")
                break
            except:
                continue

        if not comment_area:
            # 如果还是找不到，保存整个页面内容以便调试
            print("无法找到评论区，保存整个页面内容")
            comment_area = driver.find_element(By.TAG_NAME, "body")

    # 保存当前页面(第1页)评论区内容
    comment_html = comment_area.get_attribute('outerHTML')

    # 保存第1页HTML内容
    with open(os.path.join(save_dir, "page_1.html"), "w", encoding="utf-8") as f:
        f.write(comment_html)
    print(f"已保存第1页评论区内容")

    # 页面索引计数器
    page_index = 2

    # 浏览多个页面并保存内容
    pages_to_navigate = [
        "2", "3", "4", "5", "6",  # 直接点击的页码
        "下一页", "下一页", "下一页", "下一页"  # 点击下一页按钮
    ]

    # 导航到其他页面并保存评论区内容
    for page in pages_to_navigate:
        try:
            # 点击页面链接
            link = driver.find_element(By.LINK_TEXT, page)

            # 对于"下一页"按钮，有时需要双击
            if page == "下一页":
                # 首先尝试单击
                link.click()
                time.sleep(1)

                # 如果需要双击(根据原脚本中的actions.double_click)
                if "下一页" in pages_to_navigate[pages_to_navigate.index(page)+1:]:
                    actions = ActionChains(driver)
                    actions.double_click(link).perform()
            else:
                # 普通的单击操作
                link.click()

            # 等待页面加载
            time.sleep(2)

            # 保存当前页面评论区内容
            try:
                possible_selectors = [".m-review", ".m-cmt", ".review-content", "#review-content", ".comment-list"]
                comment_area = None

                for selector in possible_selectors:
                    try:
                        comment_area = driver.find_element(By.CSS_SELECTOR, selector)
                        print(f"找到评论区，使用选择器: {selector}")
                        break
                    except:
                        continue

                if not comment_area:
                    # 如果还是找不到，保存整个页面内容以便调试
                    print("无法找到评论区，保存整个页面内容")
                    comment_area = driver.find_element(By.TAG_NAME, "body")

                comment_html = comment_area.get_attribute('outerHTML')
            except Exception as e:
                print(f"获取评论区内容时出错: {e}")
                # 保存整个页面内容作为备份
                comment_html = driver.find_element(By.TAG_NAME, "body").get_attribute('outerHTML')

            # 保存当前页HTML内容
            with open(os.path.join(save_dir, f"page_{page_index}.html"), "w", encoding="utf-8") as f:
                f.write(comment_html)
            print(f"已保存第{page_index}页评论区内容")

            # 递增页面索引
            page_index += 1

        except Exception as e:
            print(f"导航到页面 {page} 时出错: {e}")

    print(f"成功保存了 {page_index-1} 个页面的评论区内容到 {save_dir} 目录")

except Exception as e:
    print(f"执行过程中出错: {e}")

finally:
    # 关闭浏览器
    driver.quit()

无法找到评论区，保存整个页面内容
已保存第1页评论区内容
无法找到评论区，保存整个页面内容
已保存第2页评论区内容
无法找到评论区，保存整个页面内容
已保存第3页评论区内容
无法找到评论区，保存整个页面内容
已保存第4页评论区内容
无法找到评论区，保存整个页面内容
已保存第5页评论区内容
无法找到评论区，保存整个页面内容
已保存第6页评论区内容
无法找到评论区，保存整个页面内容
已保存第7页评论区内容
无法找到评论区，保存整个页面内容
已保存第8页评论区内容
无法找到评论区，保存整个页面内容
已保存第9页评论区内容
无法找到评论区，保存整个页面内容
已保存第10页评论区内容
成功保存了 10 个页面的评论区内容到 icourse_pages 目录


In [10]:
import time
import pandas as pd
from playwright.sync_api import sync_playwright

def scrape_course_reviews():
    # 用于存储所有评论数据的列表
    all_reviews = []
    
    with sync_playwright() as p:
        # 启动 Chrome 浏览器
        browser = p.chromium.launch(headless=False)  # 设置 headless=True 可以在后台运行
        page = browser.new_page()
        
        # 设置浏览器窗口大小
        page.set_viewport_size({"width": 1082, "height": 672})
        
        # 打开指定的网页
        url = "https://www.icourse163.org/course/NHDX-1463126169"
        page.goto(url)
        
        # 点击"课程评价"按钮
        try:
            # 等待按钮可见并点击
            page.wait_for_selector('#review-tag-button', timeout=10000)
            page.click('#review-tag-button')
            time.sleep(3)  # 等待页面加载
        except Exception as e:
            print(f"无法点击课程评价按钮: {e}")
        
        # 定义一个函数用于提取页面上的评论内容
        def extract_reviews():
            # 等待评论元素加载
            page.wait_for_selector('div.ux-mooc-comment-course-comment_comment-list_item', timeout=5000)
            
            # 查找页面上的所有评论元素
            review_elements = page.query_selector_all('div.ux-mooc-comment-course-comment_comment-list_item')
            
            # 遍历每个评论元素
            for review_element in review_elements:
                review_data = {}
                try:
                    # 提取用户名称
                    user_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_user-info_name')
                    review_data['user'] = user_elem.text_content() if user_elem else ''
                    
                    # 提取评论时间
                    time_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time')
                    review_data['time'] = time_elem.text_content() if time_elem else ''
                    
                    # 提取评分（星星数）
                    stars = review_element.query_selector_all('.ux-icon-custom-rating-favorite')
                    review_data['rating'] = len(stars)  # 星星数表示评分
                    
                    # 提取评论内容
                    content_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_content')
                    review_data['content'] = content_elem.text_content() if content_elem else ''
                    
                except Exception as e:
                    print(f"解析评论时出错: {e}")
                
                # 将每个评论数据添加到总列表中
                if review_data:  # 只添加非空数据
                    all_reviews.append(review_data)
        
        # 定义一个函数用于点击下一页按钮
        def next_page():
            try:
                # 查找下一页按钮
                next_button_selector = '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'
                
                # 检查下一页按钮是否存在且可点击
                if page.is_visible(next_button_selector):
                    # 检查按钮是否被禁用
                    parent_li = page.query_selector('li.ux-pager_btn.ux-pager_btn__next')
                    if parent_li and 'ux-pager_btn__disabled' in parent_li.get_attribute('class'):
                        return False
                    
                    page.click(next_button_selector)
                    time.sleep(3)  # 等待页面加载
                    return True  # 成功点击下一页按钮
                else:
                    return False
            except Exception as e:
                print(f"无法点击下一页: {e}")
                return False  # 无法点击下一页按钮
        
        # 提取指定页数的评论
        num_pages = 10  # 设置要提取的页数
        for page_num in range(1, num_pages + 1):
            print(f"正在提取第 {page_num} 页的评论...")
            extract_reviews()  # 提取当前页面的评论
            
            # 如果不是最后一页，尝试翻页
            if page_num < num_pages:
                if not next_page():
                    print("已到达最后一页，停止翻页。")
                    break
        
        # 关闭浏览器
        browser.close()
    
    # 将所有提取的评论数据保存为 CSV 文件
    df = pd.DataFrame(all_reviews)
    output_file = "course_reviews1.csv"
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"数据已保存到 {output_file}")
    print("所有页面处理完毕，浏览器已关闭")
    
    return df

# 运行爬虫
if __name__ == "__main__":
    scrape_course_reviews()

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [9]:
pip install playwright

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [playwright]2 [playwright]
Note: you may need to restart the kernel to use updated packages.


In [11]:
import subprocess
import sys
import pandas as pd

def scrape_course_reviews_sync():
    """
    同步版本的爬虫，通过子进程运行避免异步冲突
    """
    
    # 创建爬虫脚本内容
    script_content = '''
import time
import pandas as pd
from playwright.sync_api import sync_playwright

# 用于存储所有评论数据的列表
all_reviews = []

with sync_playwright() as p:
    # 启动 Chrome 浏览器
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    # 设置浏览器窗口大小
    page.set_viewport_size({"width": 1082, "height": 672})
    
    # 打开指定的网页
    url = "https://www.icourse163.org/course/NHDX-1463126169"
    page.goto(url)
    
    # 点击"课程评价"按钮
    try:
        page.wait_for_selector('#review-tag-button', timeout=10000)
        page.click('#review-tag-button')
        time.sleep(3)
    except Exception as e:
        print(f"无法点击课程评价按钮: {e}")
    
    # 定义函数用于提取页面上的评论内容
    def extract_reviews():
        page.wait_for_selector('div.ux-mooc-comment-course-comment_comment-list_item', timeout=5000)
        review_elements = page.query_selector_all('div.ux-mooc-comment-course-comment_comment-list_item')
        
        for review_element in review_elements:
            review_data = {}
            try:
                user_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_user-info_name')
                review_data['user'] = user_elem.text_content() if user_elem else ''
                
                time_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_comment-info_time')
                review_data['time'] = time_elem.text_content() if time_elem else ''
                
                stars = review_element.query_selector_all('.ux-icon-custom-rating-favorite')
                review_data['rating'] = len(stars)
                
                content_elem = review_element.query_selector('.ux-mooc-comment-course-comment_comment-list_item_body_content')
                review_data['content'] = content_elem.text_content() if content_elem else ''
                
            except Exception as e:
                print(f"解析评论时出错: {e}")
            
            if review_data:
                all_reviews.append(review_data)
    
    # 定义函数用于点击下一页按钮
    def next_page():
        try:
            next_button_selector = '#comment-section > div > div.ux-mooc-comment-course-comment_pager > ul > li.ux-pager_btn.ux-pager_btn__next > a'
            
            if page.is_visible(next_button_selector):
                parent_li = page.query_selector('li.ux-pager_btn.ux-pager_btn__next')
                if parent_li and 'ux-pager_btn__disabled' in parent_li.get_attribute('class'):
                    return False
                
                page.click(next_button_selector)
                time.sleep(3)
                return True
            else:
                return False
        except Exception as e:
            print(f"无法点击下一页: {e}")
            return False
    
    # 提取指定页数的评论
    num_pages = 10
    for page_num in range(1, num_pages + 1):
        print(f"正在提取第 {page_num} 页的评论...")
        extract_reviews()
        
        if page_num < num_pages:
            if not next_page():
                print("已到达最后一页，停止翻页。")
                break
    
    # 关闭浏览器
    browser.close()

# 保存数据
df = pd.DataFrame(all_reviews)
output_file = "course_reviews1.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"数据已保存到 {output_file}")
print("所有页面处理完毕")
'''
    
    # 将脚本保存到临时文件
    with open('temp_scraper.py', 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    # 运行脚本
    try:
        result = subprocess.run([sys.executable, 'temp_scraper.py'], 
                              capture_output=True, text=True)
        print(result.stdout)
        if result.stderr:
            print("错误信息:", result.stderr)
    finally:
        # 删除临时文件
        import os
        if os.path.exists('temp_scraper.py'):
            os.remove('temp_scraper.py')
    
    # 读取生成的CSV文件
    if os.path.exists('course_reviews1.csv'):
        return pd.read_csv('course_reviews1.csv')
    else:
        print("CSV文件未生成")
        return None

# 运行函数
if __name__ == "__main__":
    df = scrape_course_reviews_sync()
    if df is not None:
        print(f"成功获取 {len(df)} 条评论")
        print(df.head())


错误信息: Traceback (most recent call last):
  File "/Users/changpengcheng/python 2/第二部分爬虫基础及相关Python的库/temp_scraper.py", line 11, in <module>
    browser = p.chromium.launch(headless=False)
  File "/opt/anaconda3/lib/python3.13/site-packages/playwright/sync_api/_generated.py", line 14475, in launch
    self._sync(
    ~~~~~~~~~~^
        self._impl_obj.launch(
        ^^^^^^^^^^^^^^^^^^^^^^
    ...<17 lines>...
        )
        ^
    )
    ^
  File "/opt/anaconda3/lib/python3.13/site-packages/playwright/_impl/_sync_base.py", line 115, in _sync
    return task.result()
           ~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/playwright/_impl/_browser_type.py", line 98, in launch
    await self._channel.send(
        "launch", TimeoutSettings.launch_timeout, params
    )
  File "/opt/anaconda3/lib/python3.13/site-packages/playwright/_impl/_connection.py", line 69, in send
    return await self._connection.wrap_api_call(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
